In [ ]:
from requests import session
import pandas as pd
MAX_SIZE = 10000
USUARIO_API = ('user-api-leitos', 'aQbLL3ZStaTr38tj')
BASE_URL = 'https://elastic-leitos.saude.gov.br/leito_ocupacao/'

In [ ]:
sessao = session()
res1 = sessao.post(BASE_URL+"_search", auth=USUARIO_API)
data = res1.json()
num_elementos = data['hits']['total']['value']
num_elementos

3738

Caso o número de elemento da consulta seja MENOR que 10000 (default) o próximo passo é suficiente para consultar os registros

In [ ]:
if(num_elementos <= MAX_SIZE):
  query = {
      "size":MAX_SIZE
  }
  response = sessao.post(BASE_URL+"_search", auth=USUARIO_API, json=query).json()
  print(len(response['hits']['hits']))

3738


Caso o número de elemento da consulta seja MAIOR que 10000 (default) siga os próximos passos:

In [ ]:
def scroll_function(scroll_id):
  scroll_query = {
      "scroll": "1m",
      "scroll_id": scroll_id
  }
  return sessao.post('https://elastic-leitos.saude.gov.br/_search/scroll', auth=USUARIO_API, json=scroll_query).json()

In [ ]:
query = {
    "size":1000
}
scroll_response = sessao.post(BASE_URL+"_search?scroll=1m", auth=USUARIO_API, json=query).json() #além do id, a lista com SIZE elementos também virá
scroll_id = scroll_response['_scroll_id']
print(scroll_id)
elements = scroll_response['hits']['hits']
print(f'Lista primeira requisição com {len(elements)}')
#O scroll de fato
sequencia_scroll = scroll_function(scroll_id)
while( len(sequencia_scroll['hits']['hits']) != 0):
  scroll_id = sequencia_scroll['_scroll_id']
  elements = elements + sequencia_scroll['hits']['hits']
  sequencia_scroll = scroll_function(scroll_id)
  print("Novo Append")
print(f'Lista Total com {len(elements)}')

DnF1ZXJ5VGhlbkZldGNoAwAAAAAAHtuGFllWTndSTm1pVEFPa19RWERRQnoxVHcAAAAAAH0gqhZOSkxvM25Pd1I4LW5YblRza09JTDJnAAAAAAB9IKsWTkpMbzNuT3dSOC1uWG5Uc2tPSUwyZw==
Lista primeira requisição com 1000
Novo Append
Novo Append
Novo Append
Lista Total com 3738
